In [ ]:
import pandas as pd
import geopandas as gpd
import mytools
import numpy as np

In [ ]:
data_4 = pd.read_excel('C:/Users/Administrator/Desktop/工参/工参/LTE小区.xlsx')

In [ ]:
data_5 = pd.read_excel('C:/Users/Administrator/Desktop/工参/工参/NR小区.xlsx')

In [ ]:
data_4_buffer = mytools.gisn.add_buffer(data_4,'经度','纬度',50)

In [ ]:
data_5_p = mytools.gisn.add_points(data_5,'经度','纬度')

In [ ]:
res = gpd.sjoin(data_4_buffer,data_5_p)

In [ ]:
data_res = pd.DataFrame()
for name , data_t in res.groupby('CGI_left'):
    data_t['方位角差异'] = np.abs(data_t['方位角_left']- data_t['方位角_right'])
    data_t['方位角差异2'] = 360-np.abs(data_t['方位角_left']- data_t['方位角_right'])
    df = data_t.loc[(data_t['方位角差异']<=55)|(data_t['方位角差异2']<=15)]
    df_use = pd.DataFrame(df.groupby(by = 'CGI_left').apply(lambda g:g['CGI_right'].str.cat(sep='/'))).reset_index()
    if df.shape[0] ==0:
        df_use = df_use.append([{'CGI_left':name,'w':np.nan}], ignore_index=True)
    else:
        pass
    df_use.columns=['4Gcgi','共覆盖cgi']
    data_res = data_res.append(df_use)

In [ ]:
data_4_rew = data_4.merge(data_res,how='left',left_on='CGI',right_on='4Gcgi')